In [ ]:
!pip install langchain openai weaviate-client tiktoken

In [ ]:
import requests
from langchain.document_loaders import TextLoader

In [ ]:
url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"

In [ ]:
res=requests.get(url)

In [ ]:
print(res.text)

In [ ]:
with open("state_of_the_union.txt","w") as f:
  f.write(res.text)

In [ ]:
loder=TextLoader('/content/state_of_the_union.txt')

In [ ]:
document=loder.load()

In [ ]:
print(document[0].page_content)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [ ]:
chunks=text_splitter.split_documents(document)

In [ ]:
print(chunks[3].page_content)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings


In [ ]:
from weaviate import Client
from weaviate.embedded import EmbeddedOptions
from langchain.vectorstores import Weaviate

In [ ]:
client=Client(
    embedded_options=EmbeddedOptions()
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 2664


In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
vectorstore=Weaviate.from_documents(
    client=client,
    documents=chunks,
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text=False
)

In [ ]:
retriever=vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

In [ ]:
query="Your question?"

In [ ]:
output_parser = StrOutputParser()

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [ ]:
rag_chain.invoke(query)